# Points of Interest

## Imports and Setup

In [44]:
%load_ext autoreload
%autoreload 2

import sys
import os

sys.path.append("../")

import logging
from pathlib import Path

from icecream import ic

from IPython.display import display
import ipywidgets as wid
from utils.ipywidgets_extended import widgets_styling

from utils.setup_notebook import init_notebook
from utils.setup_logging import setup_logging
import utils.memoize as memoize

init_notebook()
setup_logging("INFO")
memoize.set_file_store_path("harris_points_of_interest")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
import numpy as np
import pandas as pd
import scipy as sp
import numba as nb
from numba import cuda
import cv2

from utils.benchmarking import LogTimer
from utils.plotting_tools import (
    SmartFigure,
    to_ipy_image,
    plot_kernel,
    plot_matrix,
)
from utils.image_tools import load_image, LoadedImage
import utils.dyn_module as dyn

logging.getLogger("numba.cuda.cudadrv.driver").setLevel(logging.WARNING)

In [46]:
reset_memoize_store_button = wid.Button(description="Reset memoize store")
reset_memoize_store_button.on_click(lambda x: memoize.reset_store())
display(reset_memoize_store_button)


Button(description='Reset memoize store', style=ButtonStyle())

## Loading Points of Interest Implementations

In [47]:
dir_harris_impls = "./points_of_interest_impls"
harris_impls_module_names = dyn.load_modules(dir_harris_impls)


2024-11-05 16:01:58.607 INFO root Loading 0 modules started (..\utils\dyn_module.py:59)
2024-11-05 16:01:58.623 INFO root Loading 0 modules took: 19.3320 ms (..\utils\dyn_module.py:59)


## Loading Input Image Sets

In [55]:
input_image_set_dir = "./image_sets_input"
input_image_set_scaled_dir = "./image_sets_scaled_input"

input_image_sets = {}


def load_image_sets(image_sets_dir: str):
    image_sets_folders = os.listdir(image_sets_dir)
    image_sets_folders.sort()

    with LogTimer(f"Loading image sets from {Path(image_sets_dir).name}"):
        for image_set_name in image_sets_folders:
            with LogTimer(f"Loading image set {image_set_name}"):
                image_set_dir = os.path.join(image_sets_dir, image_set_name)
                image_set = []

                images_in_image_set = os.listdir(image_set_dir)
                images_in_image_set.sort()

                for image_name in images_in_image_set:
                    image = load_image(os.path.join(image_set_dir, image_name))
                    image_set.append(image)

                input_image_sets[image_set_name] = image_set


def save_image_set(image_set: list, image_set_dir: str):
    os.makedirs(image_set_dir, exist_ok=True)
    for image in image_set:
        cv2.imwrite(os.path.join(image_set_dir, image.filename), image.image_color)


load_image_sets(input_image_set_dir)
load_image_sets(input_image_set_scaled_dir)

2024-11-05 16:06:09.591 INFO root Loading image sets from image_sets_input started (notebook_cell:11)
2024-11-05 16:06:09.606 INFO root Loading image set the_office started (notebook_cell:13)
2024-11-05 16:06:09.633 INFO root Loading 20241105_124827.jpg started (..\utils\image_tools.py:69)
2024-11-05 16:06:10.081 INFO root Loading 20241105_124827.jpg took: 458.2781 ms (..\utils\image_tools.py:69)
2024-11-05 16:06:10.099 INFO root Loading 20241105_124828.jpg started (..\utils\image_tools.py:69)
2024-11-05 16:06:10.672 INFO root Loading 20241105_124828.jpg took: 558.4061 ms (..\utils\image_tools.py:69)
2024-11-05 16:06:10.715 INFO root Loading 20241105_124831.jpg started (..\utils\image_tools.py:69)
2024-11-05 16:06:11.504 INFO root Loading 20241105_124831.jpg took: 806.5266 ms (..\utils\image_tools.py:69)
2024-11-05 16:06:11.524 INFO root Loading 20241105_124833.jpg started (..\utils\image_tools.py:69)
2024-11-05 16:06:12.519 INFO root Loading 20241105_124833.jpg took: 980.9793 ms (..\u

## Image set scaler

In [49]:
def get_largest_image_in_image_set(image_set: list) -> np.array:
    largest_image = None
    largest_pixel_count = 0
    for image in image_set:
        resolution = image.image_color.shape[:2]
        pixel_count = resolution[0] * resolution[1]
        if pixel_count > largest_pixel_count:
            largest_pixel_count = pixel_count
            largest_image = image
    return largest_image


def get_largest_resolution_in_image_set(image_set: list) -> tuple:
    largest_image = get_largest_image_in_image_set(image_set)
    return largest_image.image_color.shape[:2]

In [ ]:
KEY_SCALER_IMAGE_SET_DROPDOWN = "scaler_image_set_dropdown"
scaler_image_set_dropdown = wid.Dropdown(
    options=list(input_image_sets.keys()),
    value=memoize.get(
        KEY_SCALER_IMAGE_SET_DROPDOWN,
        default=next(iter(input_image_sets.keys())),
        possible_values=input_image_sets.keys(),
    ),
    description="Image set",
    **widgets_styling,
)
scaler_largest_resolution_label = wid.Label("Largest image: (X,X)")
KEY_SCALER_SCALE_SLIDER = "scaler_scale_slider"
scaler_scale_slider = wid.FloatSlider(
    value=memoize.get(KEY_SCALER_SCALE_SLIDER, default=1.0),
    min=0.1,
    max=30.0,
    step=0.1,
    continuous_update=True,
    orientation="horizontal",
    readout=True,
    readout_format=".1f",
    description="Scale",
)
scaler_result_resolution_label = wid.Label("Resulting largest image: (X,X)")
scaler_create_scaled_image_set_button = wid.Button(
    description="Create scaled image set",
    **widgets_styling,
)


def on_update_resolution_labels(change=None):
    memoize.set(KEY_SCALER_IMAGE_SET_DROPDOWN, scaler_image_set_dropdown.value)
    memoize.set(KEY_SCALER_SCALE_SLIDER, scaler_scale_slider.value)

    image_set = input_image_sets[scaler_image_set_dropdown.value]
    largest_image = get_largest_image_in_image_set(image_set)
    largest_resolution = largest_image.image_color.shape[:2]
    scaler_largest_resolution_label.value = f"Largest image: {largest_resolution}"
    scaler_result_resolution_label.value = f"Resulting largest image: {tuple(int(x * 1/scaler_scale_slider.value) for x in largest_resolution)}"


scaler_scale_slider.observe(on_update_resolution_labels, names="value")
scaler_image_set_dropdown.observe(on_update_resolution_labels, names="value")
on_update_resolution_labels()


def create_scaled_image_set(change=None):
    with LogTimer(f"Creating scaled image set for {scaler_image_set_dropdown.value}"):
        original_image_set = input_image_sets[scaler_image_set_dropdown.value]

        scale = 1 / scaler_scale_slider.value

        largest_resolution = get_largest_resolution_in_image_set(original_image_set)
        scaled_resolution = tuple(int(x * scale) for x in largest_resolution)
        scaled_resolution_fs_string = f"({scaled_resolution[0]},{scaled_resolution[1]})"

        scaled_image_set_name = (
            f"{scaler_image_set_dropdown.value}_scaled_{scaled_resolution_fs_string}"
        )

        scaled_image_set_dir = os.path.join(
            input_image_set_scaled_dir, scaled_image_set_name
        )

        new_image_set = []
        for image in original_image_set:
            scaled_resolution = tuple(
                int(x * scale) for x in image.image_color.shape[:2]
            )
            scaled_resolution_fs_string = (
                f"({scaled_resolution[0]},{scaled_resolution[1]})"
            )
            with LogTimer(
                f"Resizing image {image.filename} from {image.image_color.shape[:2]} to {scaled_resolution}"
            ):
                new_image = LoadedImage()
                new_image.image_color = cv2.resize(image.image_color, scaled_resolution)
                new_image.filename = f"{Path(image.filename).stem}_{scaled_resolution_fs_string}{Path(image.filename).suffix}"
                new_image_set.append(new_image)

        save_image_set(new_image_set, scaled_image_set_dir)
        load_image_sets(input_image_set_scaled_dir)


scaler_create_scaled_image_set_button.on_click(create_scaled_image_set)

display(
    wid.VBox(
        [
            wid.HBox([scaler_image_set_dropdown, scaler_largest_resolution_label]),
            wid.HBox([scaler_scale_slider, scaler_result_resolution_label]),
            scaler_create_scaled_image_set_button,
        ]
    )
)

2024-11-05 16:09:35.454 INFO root Creating scaled image set for the_office started (notebook_cell:49)
2024-11-05 16:09:35.473 INFO root Resizing image 20241105_124827.jpg from (8160, 6120) to (604, 453) started (notebook_cell:74)
2024-11-05 16:09:35.505 INFO root Resizing image 20241105_124827.jpg from (8160, 6120) to (604, 453) took: 20.0889 ms (notebook_cell:74)
2024-11-05 16:09:35.532 INFO root Resizing image 20241105_124828.jpg from (8160, 6120) to (604, 453) started (notebook_cell:74)
2024-11-05 16:09:35.567 INFO root Resizing image 20241105_124828.jpg from (8160, 6120) to (604, 453) took: 28.8506 ms (notebook_cell:74)
2024-11-05 16:09:35.582 INFO root Resizing image 20241105_124831.jpg from (8160, 6120) to (604, 453) started (notebook_cell:74)
2024-11-05 16:09:35.615 INFO root Resizing image 20241105_124831.jpg from (8160, 6120) to (604, 453) took: 17.2515 ms (notebook_cell:74)
2024-11-05 16:09:35.639 INFO root Resizing image 20241105_124833.jpg from (8160, 6120) to (604, 453) st

## Running Points of Interest detection

In [ ]:
fig_width = 16
image_size = 512
